## 一、配置环境变量

In [1]:
import os
from dotenv import dotenv_values

In [2]:
env_variables = dotenv_values('.env')
for var in env_variables:
    print(var)
    os.environ[var] = env_variables[var]

ZHIPUAI_API_KEY
DASHSCOPE_API_KEY


In [3]:
os.getenv("DASHSCOPE_API_KEY")

'sk-f7e5ff2257cd4cff95eaedffb9ddb35e'

## 二、配置LLM

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
from typing import Dict, List, Optional, Tuple, Union

from zhipuai import ZhipuAI
import dashscope
import random
from http import HTTPStatus
import dashscope
from dashscope import Generation  # 建议dashscope SDK 的版本 >= 1.14.0

PROMPT_TEMPLATE = dict(
    RAG_PROMPT_TEMPALTE="""使用以上下文来回答用户的问题。如果你不知道答案，就说你不知道。总是使用中文回答。
        问题: {question}
        可参考的上下文：
        ···
        {context}
        ···
        如果给定的上下文无法让你做出回答，请回答数据库中没有这个内容，你不知道。
        有用的回答:""",
    InternLM_PROMPT_TEMPALTE="""先对上下文进行内容总结,再使用上下文来回答用户的问题。如果你不知道答案，就说你不知道。总是使用中文回答。
        问题: {question}
        可参考的上下文：
        ···
        {context}
        ···
        如果给定的上下文无法让你做出回答，请回答数据库中没有这个内容，你不知道。
        有用的回答:"""
)


class BaseModel:
    def __init__(self, path: str = '') -> None:
        self.path = path

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        pass

    def load_model(self):
        pass

class GLM4Chat(BaseModel):
    def __init__(self, path: str = '', model: str = "glm-4") -> None:
        super().__init__(path)
        self.model = model

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        client = ZhipuAI(api_key=os.getenv("ZHIPUAI_API_KEY"))  # 填写您自己的APIKey
        history.append({'role': 'user', 'content': PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content)})
        response = client.chat.completions.create(
            model="glm-4",  # 填写需要调用的模型名称
            messages=history
        )
        return response.choices[0].message

class QwenChat(BaseModel):
    def __init__(self, path: str = '', model: str = "qwen-turbo") -> None:
        super().__init__(path)
        dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")
        self.model = model

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        history.append({'role': 'user', 'content': PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content)})
        response = Generation.call(model="qwen-turbo",
                                messages=history,
                                # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                seed=random.randint(1, 10000),
                                # 将输出设置为"message"格式
                                result_format='message')
        if response.status_code == HTTPStatus.OK:
            return response.output.choices[0].message["content"]
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))


class OpenAIChat(BaseModel):
    def __init__(self, path: str = '', model: str = "gpt-3.5-turbo-1106") -> None:
        super().__init__(path)
        self.model = model

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        from openai import OpenAI
        client = OpenAI()   
        history.append({'role': 'user', 'content': PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content)})
        response = client.chat.completions.create(
            model=self.model,
            messages=history,
            max_tokens=150,
            temperature=0.1
        )
        return response.choices[0].message.content

class InternLMChat(BaseModel):
    def __init__(self, path: str = '') -> None:
        super().__init__(path)
        self.load_model()

    def chat(self, prompt: str, history: List = [], content: str='') -> str:
        prompt = PROMPT_TEMPLATE['InternLM_PROMPT_TEMPALTE'].format(question=prompt, context=content)
        response, history = self.model.chat(self.tokenizer, prompt, history)
        return response


    def load_model(self):
        import torch
        from transformers import AutoTokenizer, AutoModelForCausalLM
        self.tokenizer = AutoTokenizer.from_pretrained(self.path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(self.path, torch_dtype=torch.float16, trust_remote_code=True).cuda()


In [5]:
qwenchat = QwenChat()
qwenchat.chat(history=[], prompt="hello", content="")

'你好！有什么我可以帮助你的吗？'

## 三、设置知识库FaissVectorStore

In [6]:
import os
import time
from typing import Dict, List, Optional, Tuple, Union
import json

import faiss

from RAG.Embeddings import BaseEmbeddings, OpenAIEmbedding, JinaEmbedding, ZhipuEmbedding
import numpy as np
from tqdm import tqdm

from RAG.VectorBase import VectorStore


class FaissVectorStore(VectorStore):

    def __init__(self, document: List[str] = ['']) -> None:
        super().__init__(document)
        self.document = document

    def get_vector(self, EmbeddingModel: BaseEmbeddings) -> List[List[float]]:
        self.vectors = []
        for doc in tqdm(self.document, desc="Calculating embeddings"):
            self.vectors.append(EmbeddingModel.get_embedding(doc))
        return self.vectors

    def persist(self, path: str = 'storage'):
        if not os.path.exists(path):
            os.makedirs(path)
        with open(f"{path}/doecment.json", 'w', encoding='utf-8') as f:
            json.dump(self.document, f, ensure_ascii=False)
        if self.vectors:
            with open(f"{path}/vectors.json", 'w', encoding='utf-8') as f:
                json.dump(self.vectors, f)

    def load_vector(self, path: str = 'storage'):
        with open(f"{path}/vectors.json", 'r', encoding='utf-8') as f:
            self.vectors = json.load(f)
        with open(f"{path}/doecment.json", 'r', encoding='utf-8') as f:
            self.document = json.load(f)

    def get_similarity(self, vector1: List[float], vector2: List[float]) -> float:
        return BaseEmbeddings.cosine_similarity(vector1, vector2)

    def generateIndexFlatL2(self):
        dim, measure = 768, faiss.METRIC_L2
        param = 'Flat'
        vectors = np.array([vector for vector in self.vectors]).astype('float32')
        index = faiss.index_factory(dim, param, measure)
        index.add(vectors)  # 将向量库中的向量加入到index中
        return index


    def query(self, query: str, EmbeddingModel: BaseEmbeddings, k: int = 1) -> List[str]:

        start_time = time.time()
        index = self.generateIndexFlatL2()
        end_time = time.time()
        print(' 构建索引 cost %f second' % (end_time - start_time))
        query_vector = np.array([EmbeddingModel.get_embedding(query)]).astype("float32")

        end_time = time.time()
        D, I = index.search(query_vector, k)  # xq为待检索向量，返回的I为每个待检索query最相似TopK的索引list，D为其对应的距离
        print(' 检索 cost %f second' % (time.time() - end_time))
        return np.array(self.document)[I[0,:]].tolist()

In [7]:
vector = FaissVectorStore()

In [8]:
vector.load_vector('../../storage/history_24_1') # 加载本地的数据库

In [9]:
len(vector.vectors[0])

768

## 四、定义embedding算法

In [10]:
from RAG.PaddleEmbedding import PaddleEmbedding
embedding = PaddleEmbedding() # 创建EmbeddingModel

/workspace/anaconda3/envs/paddlenlp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/anaconda3/envs/paddlenlp/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2024-06-25 23:07:15,803] [    INFO] - We are using (<class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'>, False) to load 'rocketqa-zh-base-query-encoder'.
[2024-06-25 23:07:15,803] [    INFO] - Already cached /workspace/liujing/.paddlenlp/models/rocketqa-zh-base-query-encoder/ernie_3.0_base_zh_vocab.txt
[2024-06-25 23:07:15,833] [    INFO] - tokenizer config file saved in /workspace/liujing/.paddlenlp/models/rocketqa-zh-base-query-encoder/tokenizer_config.json
[2024-06-25 23:07:15,834] [    INFO] - Special tokens fi

In [11]:
question = '介绍一下秦二世'
content = vector.query(question, EmbeddingModel=embedding, k=3)[0]
print(f'知识库输出：{content}')
chat = QwenChat()
print(chat.chat(question, [], content))

 构建索引 cost 0.373451 second
 检索 cost 0.043894 second
知识库输出：考察周朝各诸侯国的先后关系，作《十二诸侯年表》第二。
　　春秋以后，陪臣执政，强国之君竞相称王，及至秦王嬴政，终于吞并各国，铲除封地，独享尊号。
　　作《六国年表》第三。
　　秦帝暴虐，楚人陈胜发难，项氏大乱秦朝，汉王仗义征伐。
　　八年之间，天下三易其主，事变繁多，所以详著《秦楚之际月表》第四。
　　汉朝兴建以来，直到太初一百年间，诸侯废立分割的情况，谱录记载不明，史官也无法继续记载，但可据其世系推知其强弱的原由。　　作《汉兴以来诸侯年表》第五。
　　高祖始取天下，辅佐他创业的功臣，都得到剖符封爵，恩泽传给他们的后世子孙；有的忘其亲疏远近，分不出辈份，有的竟至杀身亡国。
　　作《高祖功臣侯者年表》第六。
　　惠帝、景帝年间，增封功臣宗属爵位和食邑。
　　作《惠、景间侯者年表》第七。
　　北面攻打强悍的匈奴，南面诛讨强劲的越人，征讨四方夷蛮，不少人以武功封侯。
　　作《建元以来侯者年表》第八。
　　诸侯国日渐强大，吴楚等七国南北连成一片，诸侯王子弟众多，没有爵位封邑，朝廷下令推行恩义，分封诸侯王子弟为侯，致使王国势力日益削弱，而德义却归于朝廷。
　　作《王子侯者年表》第九。
　　国家的贤相良将，是民众的表率。
　　曾看到汉兴以来将相名臣年表，对贤者则记其治绩，对不贤者则明其劣迹。
　　作《汉兴以来将相名臣年表》第十。
　　夏、商、周三代之礼，各有增减，但总的来看其要领都在于使礼切近人性，通于王道，所以礼根据人的质朴本性而制成，去掉那些繁文缛节，大体顺应了古今之变。

　　作《礼书》第一。
　　乐是用来移风易俗的。
　　自《雅》、《颂》之声兴起，人们就已经喜好郑卫之音，由来已久了。
　　被人情所感发，那远方异域之人，就会归附。
　　仿照已有《乐书》来论述自古以来音乐的兴衰。
　　作《乐书》第二。
　　史记没有军队，国家不会强盛；没有德政，国家就不会兴旺。
　　黄帝、商汤、周武王以明于此而兴，夏桀、商纣、秦二世以昧于此而亡，怎么可以对此不慎重呢？《司马法》产生已很久了。
　　姜太公、孙武、吴起、王子成甫能继承并有所发明，切合近世情况，极尽人世之变。
　　作《律书》第三。
　　乐律处于阴而治阳，历法处于阳而治阴，律历交替相治，其间不允许丝毫差错。
　　原有五